In [ ]:
#Creating a pair RDD
trafficpath = "file:///home/shubham/Downloads/nayadata/Dodgers.data"
gamespath = "file:///home/shubham/Downloads/nayadata/Dodgers.events"

In [ ]:
#creating an RDD
traffic = sc.textFile(trafficpath)
traffic.take(10)

#As we can see there's a slice of 05 minutes and the second column depicts number of cars passed by

In [ ]:
games = sc.textFile(gamespath)
games.take(10)
#We have start and end time for a game
#We also have audiences, opponents, Win/Loss with score

In [ ]:
from datetime import datetime
import csv
from StringIO import StringIO

#We will setup a function to make a tuple of two elements i.e. data and 5 minute time slice

def parsetraffic(row):
    date_fmt= "%m/%d/%Y %H:%M"
    row = row.split(",")
    row[0] = datetime.strptime(row[0],date_fmt)
    row[1] = int(row[1])
    return (row[0],row[1])

In [ ]:
trafficparsed = traffic.map(parsetraffic)
#trafficparsed.take(10)

In [ ]:
trafficparsed.take(10)


In [ ]:
#Summerazing a pair RDD
#Each date occurs multiple times & thus values need to be sum
dailytrend = trafficparsed.map(lambda x:(x[0].date(), x[1]))\
                        .reduceByKey(lambda x,y:x+y) #This will return a RDD where each record is date 


In [ ]:
dailytrend.take(10)

In [ ]:
#Now to sort the data by the values and see which are the days on which we had maximum traffic

dailytrend.sortBy(lambda x:-x[1]).take(10)

#TO get our data sorted in descending order we hab=ve to take negative of total number of cars
#With top 10 records we can see that we have the date with maximum traffic

In [ ]:
#Now it will be interesting to see whether the most traffic days had games or not on that particular day
#To do this we need to murge our dataset that has dates of games & for that we need a join operation

#Joining with games

def parsegames(row):
    date_fmt = "%m/%d/%y"
    row = row.split(",")
    row[0] = datetime.strptime(row[0],date_fmt).date()
    return (row[0],row[4])




In [ ]:

gamesparsed = games.map(parsegames)
#gamesparsed.first()
#print(gamesparsed)
#gamesparsed.first()

In [ ]:
#gamesparsed.first()
gamesparsed.take(10)

In [ ]:
#Joining RDD with our original RDD

dailytrendcombined = dailytrend.leftOuterJoin(gamesparsed)

In [ ]:
dailytrendcombined.take(10)

#Key is still a date but the value is now a tuple which contains values from 1st RDD & 2nd RDD.
#Whenever there's no match for 2nd RDD we have its corresponding value as None
#the reason to choose left outer join so that we can keep all dates in our dataset whether or not there's a game that day.

In [ ]:
#This function will take each record, convert it into a tuple of 4 elements i.e. Date, Oponent, Type of Day, No. of cars

def checkgameday(row):
    if row[1][1] == None:
        return (row[0],row[1][1],"Regular Day",row[1][0])
    else:
        return (row[0],row[1][1],"Game Day",row[1][0])

In [ ]:
dailytrendbygames = dailytrendcombined.map(checkgameday)

In [ ]:
dailytrendbygames.take(10)

In [ ]:
# We need to use sort by operation again but this time sorting by the forth value each time in each record.

dailytrendbygames.sortBy(lambda x: -x[3]).take(10)

#Now we have top 10days of traffic with game held or not

In [ ]:
#Now we need to see the average traffic on days with game and without game for this we need 
#A combined by key operation

#Basically a 'CombinedByKey' consist of three functions i.e. create combiner function, Merge function & MergeCombiner function.

#Create Combiner Function : Initializes a value when a key is seen for the first time within a partition.
#Merge Function : Specifies how values with the smekeyshould be combined for each record which are present within a single partition
#The result from each partition are then combined using a 'MergeCombiner Function'

In [ ]:
#Average on Game day vs Non-Game Day

dailytrendbygames.map(lambda x:(x[2],x[3]))\
                .combineByKey(lambda value : (value, 1),\
            lambda acc,value:(acc[0]+value,acc[1]+1),\
            lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1]))\
            .mapValues(lambda x:x[0]/x[1])\
            .collect()